<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering on MIND Dataset.

Neural Collaborative Filtering (NCF) is a well known recommendation algorithm that generalizes the matrix factorization problem with multi-layer perceptron. 

This notebook provides an example of how to utilize and evaluate NCF implementation in the `recommenders`. We use a smaller dataset in this example to run NCF efficiently with GPU acceleration on a [Data Science Virtual Machine](https://azure.microsoft.com/en-gb/services/virtual-machines/data-science-virtual-machines/).

In [3]:
%load_ext autoreload
%autoreload 2

[autoreload of psutil failed: Traceback (most recent call last):
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/psutil/__init__.py", line 251, in <module>
    raise ImportError(msg)
ImportError: version conflict: '/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/psutil/_psutil_osx.cpython-39-darwin.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of numpy.ma.core failed: Traceback (most recent call last):
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 335, in update_class
    if (old_obj == new_obj) is True:
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/numpy/ma/core.py", line 4232, in __eq__
    """
  File "/opt/anaconda3/envs/recommPaper/lib/python3.9/site-packages/numpy/ma/core.py", line 4160, in _comparison
    # Cast fill value to bool_ if needed. If it canno

In [3]:
import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
# from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.12.4 (main, Jun 16 2024, 01:58:47) [Clang 15.0.0 (clang-1500.3.9.4)]
Pandas version: 2.2.3
Tensorflow version: 2.18.0


Set the default parameters.

In [6]:
# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

### 1. Load the MIND dataset

In [8]:
behaviors_df = pd.read_csv("TaFR/data/MIND/MINDsmall_train/behaviors.tsv", sep="\t", 
                           names=["impression_id", "user_id", "timestamp", "history", "impressions"])

# little preprocessing to get the interactions
def process_impressions(row):
    impressions = row["impressions"].split(" ")
    return [(row["user_id"], imp.split("-")[0], int(imp.split("-")[1]), row["timestamp"]) 
            for imp in impressions]

interactions = []
for _, row in behaviors_df.iterrows():
    interactions.extend(process_impressions(row))

interactions_df = pd.DataFrame(interactions, columns=["userID", "itemID", "rating", "timestamp"])

news_df = pd.read_csv("TaFR/data/MIND/MINDsmall_train/news.tsv", sep="\t", 
                      names=["news_id", "category", "subcategory", "title", "abstract", "url", "entity_list", "relation_list"])

interactions_df = interactions_df[interactions_df["itemID"].isin(news_df["news_id"])]

# convert to csv
interactions_df.to_csv("mind_interactions_train.csv", index=False)

### 2. Split the data using the Spark chronological splitter provided in utilities

In [ ]:
## -*- not required in our dataset -*
# train, test = python_chrono_split(df, 0.75)

Write datasets to csv files.

In [13]:
train_file = "mind_interactions_train.csv"
# train.to_csv(train_file, index=False)
# test_file = "./test.csv"
# test.to_csv(test_file, index=False)

Generate an NCF dataset object from the data subsets.

In [15]:
data = NCFDataset(train_file=train_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing mind_interactions_train.csv ...


ValueError: invalid literal for int() with base 10: 'U13740'

### 3. Train the NCF model on the training data, and get the top-k recommendations for our testing data

NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated. 

In [ ]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [11]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [6.31s]: train_loss = 0.259318 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [6.28s]: train_loss = 0.246134 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [6.21s]: train_loss = 0.240125 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [6.23s]: train_loss = 0.235913 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [6.31s]: train_loss = 0.232268 


Took 317.7864 seconds for training.


In the movie recommendation use case scenario, seen movies are not recommended to the users.

In [12]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 2.7835 seconds for prediction.


### 4. Evaluate how well NCF performs

The ranking metrics are used for evaluation.

In [13]:
eval_map = map(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.049650
NDCG:	0.200524
Precision@K:	0.183033
Recall@K:	0.102721


NDCG - Normalised Discounted Cumulative Gain  
MAP - Mean Average Precision

In [ ]:
# Record results for tests - ignore this cell
store_metadata("map", eval_map)
store_metadata("ndcg", eval_ndcg)
store_metadata("precision", eval_precision)
store_metadata("recall", eval_recall)
store_metadata("train_time", train_time.interval)
store_metadata("test_time", test_time.interval)